# Visualizing with Tableau
### Full scrape data with bootstrapped GPT2
---

In [1]:
# libraries
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, ElasticNetCV, RidgeCV, LassoCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split, cross_val_score ## ** GridSearchCV
from sklearn.metrics import confusion_matrix as cm

In [2]:
# data
ss_scrape = pd.read_csv('data/ss_data.csv', low_memory=False)
gpt2_scrape = pd.read_csv('data/gp_data.csv', low_memory=False)
all_scrape = pd.concat([ss_scrape, gpt2_scrape], sort=False)   # combine

In [19]:
df_boot = pd.read_csv('data/bootstrapped_gp.csv', low_memory=False).drop(columns='Unnamed: 0')
df_boot.head(3)

,class,score,num_crossposts,title_selftext
0,0.0,29,NaN,"Riot we supported you for almost 5 years, we n..."
1,0.0,10,NaN,Enjoy!. We were camping and my 15 year old in ...
2,0.0,56,NaN,"Here in my neighborhood beat cancer, so her pa..."


In [20]:
df_boot['class'].value_counts()

1.0    32293
0.0    32293
Name: class, dtype: int64

#### CVEC for word frequences

In [21]:
cvec = CountVectorizer(max_features=10000, ngram_range=(1,3))

df_cv = cvec.fit_transform(df_boot['title_selftext'])
df_cv = pd.DataFrame(df_cv.todense(), columns=cvec.get_feature_names())

In [22]:
df_boot[df_boot['class']==0]

,class,score,num_crossposts,title_selftext
0,0.0,29,NaN,"Riot we supported you for almost 5 years, we n..."
1,0.0,10,NaN,Enjoy!. We were camping and my 15 year old in ...
2,0.0,56,NaN,"Here in my neighborhood beat cancer, so her pa..."
3,0.0,1268,NaN,[NSFW] Get ready for a Scottish soft drink
4,0.0,38,NaN,Fear is the story of how I could think about w...
...,...,...,...,...
32288,0.0,6,0.0,[Homemade] Green eggs and vegetables for dinner
32289,0.0,5,0.0,Is this a good overclocking speed for 2400g fo...
32290,0.0,7,0.0,"I just turned 31, too soon to be nice to rekin..."
32291,0.0,4,0.0,People are pooping more than 1500 homes have b...


In [25]:
gp_cv = df_cv.loc[32293:] # separate classes
ss_cv = df_cv.loc[:32292]

### Word frequencies

In [26]:
# get all words by frequency
ss_freq = ss_cv.sum().sort_values(ascending=False)
gp_freq = gp_cv.sum().sort_values(ascending=False)

### Get word frequencies DF

In [29]:
n=2000
# list top n words from each subreddit by frequency
ss_top = list(ss_freq[:n].index)
gp_top = list(gp_freq[:n].index)
top_all = set(ss_top+gp_top)

# make df of top words by frequncy
df_freq = pd.DataFrame([{word: ss_freq[word] for word in top_all if word in ss_freq.index},
                        {word: gp_freq[word] for word in top_all if word in gp_freq.index}
                       ]).T.fillna(0).reset_index().rename(columns={'index' : 'word',
                                                                    0:'SubredditSimulator',
                                                                    1:'SubSimulatorGPT2',})
df_freq[df_freq._get_numeric_data().columns] = df_freq._get_numeric_data().astype('int')
df_freq.to_csv('data/df_freq.csv')
df_freq.head(3)
len(df_freq)

2496

In [30]:
df_freq.head(3)

,word,SubredditSimulator,SubSimulatorGPT2
0,quickly,119,100
1,in the world,174,405
2,he would,100,480


### Define stopwords

In [31]:
nltk_stopwords = set(stopwords.words('english'))

def check_stopwords(x):
    grams = x.split(' ')
    stop = 0
    for gram in grams:
        if gram in nltk_stopwords:
            stop += 1
    if stop==len(grams):
        return 'stopword'
    elif stop>0:
        return 'part-stopword'
    else:
        return 'non-stopword'

In [32]:
df_freq['stopword'] = df_freq['word'].apply(check_stopwords)
df_freq.head(4)

,word,SubredditSimulator,SubSimulatorGPT2,stopword
0,quickly,119,100,non-stopword
1,in the world,174,405,part-stopword
2,he would,100,480,part-stopword
3,but don,317,1298,stopword


In [33]:
df_freq['stopword'].value_counts()

non-stopword     1158
stopword          687
part-stopword     651
Name: stopword, dtype: int64

### Define ngrams

In [34]:
def ngram_checker(word):
    ngram = 1
    for char in word:
        if char == ' ': ngram+=1
    return ngram

In [35]:
df_freq['ngrams'] = df_freq['word'].apply(ngram_checker)
df_freq.head(3)

,word,SubredditSimulator,SubSimulatorGPT2,stopword,ngrams
0,quickly,119,100,non-stopword,1
1,in the world,174,405,part-stopword,3
2,he would,100,480,part-stopword,2


### Add mean TF-IDF scores
Average of scores across corpus when `tf-idf > 0.1`

In [36]:
tfidf_df = pd.read_csv('data/top_tfidf.csv').set_index('Unnamed: 0')

In [37]:
def get_tfidf_mean(word):
    if word in tfidf_df.index:
        return tfidf_df.loc[word].mean()

In [38]:
df_freq['tf-idf mean'] = df_freq['word'].apply(get_tfidf_mean)
df_freq.head(3)

,word,SubredditSimulator,SubSimulatorGPT2,stopword,ngrams,tf-idf mean
0,quickly,119,100,non-stopword,1,NaN
1,in the world,174,405,part-stopword,3,0.221272
2,he would,100,480,part-stopword,2,NaN


### Get MnNB coefs

In [40]:
# combine dfs, define X, y
X = df_boot['title_selftext']
y = df_boot['class']

# tfidf transform
tfid = TfidfVectorizer(max_features=10000, ngram_range=(1,3),)
X = tfid.fit_transform(X)
X = pd.DataFrame(X.toarray(), columns=tfid.get_feature_names())

# keep only cols from df_freq
Xf = X[[w for w in df_freq['word'] if w in X.columns]]

In [41]:
mnb = MultinomialNB(alpha=1*np.e**-20) # approaching 0 appears to be the best alpha
mnb.fit(Xf, y)

df_freq['mnb coef'] = mnb.coef_[0]
df_freq.head()

,word,SubredditSimulator,SubSimulatorGPT2,stopword,ngrams,tf-idf mean,mnb coef
0,quickly,119,100,non-stopword,1,NaN,-10.101875
1,in the world,174,405,part-stopword,3,0.221272,-7.894009
2,he would,100,480,part-stopword,2,NaN,-8.565435
3,but don,317,1298,stopword,2,NaN,-7.656018
4,needs,205,337,non-stopword,1,0.249849,-8.510327


### Get Ridge coefs

In [42]:
lr = LogisticRegressionCV(Cs=np.logspace(.1, 1, 50), penalty='l2', solver='lbfgs', max_iter=5000, cv=3)
lr.fit(Xf, y)
df_freq['ridge tf-idf coefficient'] = lr.coef_[0] # new col for coef
print(cross_val_score(lr, Xf, y, cv=5).mean())
df_freq.head(3)

KeyboardInterrupt: 

#### Add `Total Count`,  `Ridge β (direction)` and `Ridge β (abs)`

In [ ]:
df_freq['Total Count'] = df_freq['SubredditSimulator'] + df_freq['SubSimulatorGPT2']

def sway(x):
    if x<0: return 'β < 0 (ss)'
    else: return 'β > 0 (gpt2)'  
df_freq['Ridge β'] = df_freq['ridge tf-idf coefficient'].apply(sway)
df_freq['Ridge β (abs)'] = df_freq['ridge tf-idf coefficient'].abs()
df_freq

### Get doc (appearance) counts:

In [24]:
doc_count = pd.DataFrame(df_freq['word'])

for i in doc_count.index:
    word = doc_count.loc[i, 'word']
    try:
        doc_count.loc[i, 'ss_doc_count'] = len(ss_cv[ss_cv[word]!=0])
    except:
        print(i)

In [26]:
for i in doc_count.index:
    word = doc_count.loc[i, 'word']
    try:
        doc_count.loc[i, 'gp_doc_count'] = len(gp_cv[gp_cv[word]!=0])
    except:
        print(i)
doc_count.head(3)

,word,ss_doc_count,gp_doc_count
0,shoot,4.0,1.0
1,dank maymays,2.0,0.0
2,my dog,2.0,4.0


In [27]:
df_freq = pd.concat([df_freq, doc_count.drop(columns='word')], axis=1)
df_freq.head(3)

,word,SubredditSimulator,SubSimulatorGPT2,stopword,ngrams,tf-idf mean,mnb coef,ridge tf-idf coefficient,Total Count,Ridge β,Ridge β (abs),ss_doc_count,gp_doc_count
0,shoot,4,1,non-stopword,1,0.190601,-9.711637,-0.804550,5,β < 0 (ss),0.804550,4.0,1.0
1,dank maymays,2,0,non-stopword,2,NaN,-27.768190,-0.516721,2,β < 0 (ss),0.516721,2.0,0.0
2,my dog,2,4,part-stopword,2,0.329556,-7.242680,1.072602,6,β > 0 (gpt2),1.072602,2.0,4.0


## Export

In [28]:
from better_profanity import profanity
profanity.load_censor_words()

In [29]:
def limit_profanity(x):
    grams = x.split(' ')
    res = []
    for g in grams:
        if profanity.contains_profanity(g):
            if 'ing' not in g: res.append("".join([g[0]]+['*']*(len(g)-2)+[g[-1]]))
            else: res.append("".join([g[0]]+['*']*(len(g)-4)+[g[-3]]))
        else:
            res.append(g)
    return " ".join(res)

In [31]:
df_freq.loc[np.where([profanity.contains_profanity(word) for word in df_freq['word']])].head(3)

,word,SubredditSimulator,SubSimulatorGPT2,stopword,ngrams,tf-idf mean,mnb coef,ridge tf-idf coefficient,Total Count,Ridge β,Ridge β (abs),ss_doc_count,gp_doc_count
6,of shit you,0,3,part-stopword,3,0.200330,-9.322210,0.089508,3,β > 0 (gpt2),0.089508,0.0,1.0
56,gay and,0,3,part-stopword,2,0.199115,-8.002887,0.650739,3,β > 0 (gpt2),0.650739,0.0,3.0
75,ass tittyfuck bitch,3,0,non-stopword,3,0.269730,-27.768190,-0.364826,3,β < 0 (ss),0.364826,1.0,0.0


In [32]:
df_freq['word'] = df_freq['word'].apply(limit_profanity)

In [34]:
df_freq.to_csv('df_freq.csv', index=False)